In [10]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.pipeline import Pipeline
from sklearn import set_config
from matplotlib import style
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler 
from sklearn.cluster import KMeans

In [26]:
import models
hankel = models.Hankel()
rank = models.Rank()
cluster = models.Cluster()
meepc = models.MEEPC() 

In [34]:
df = pd.read_csv('~/data/training_data.csv')

In [35]:
df

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
1,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
2,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
3,167.892,0.625471,11.6175,504.140,0.293168,0.001107,62.1436,1.0,1.0,1.0,...,0.0,8279.1,14.35530,238.111,0.620147,0.667695,67.0974,60.0435,0.11,1.0
4,167.895,0.619473,11.6179,504.145,0.293009,0.001080,62.1472,1.0,1.0,1.0,...,0.0,8279.1,14.00090,264.802,0.620240,0.672394,67.1004,60.1557,0.11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971995,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971996,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971997,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0
971998,179.618,0.625473,11.9156,420.420,0.228225,1.903020,43.3472,2.0,1.0,1.0,...,0.0,8279.1,5.21471,783.722,0.536200,0.627771,64.5326,86.5871,1.47,-1.0


In [36]:
df_normal=df[df['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_attack=df[df['Attack LABLE (1:No Attack, -1:Attack)']==-1]

In [37]:
# sensors = [3,9,25,43]
sensors = [0,7,14,57,58,59,60,61,63]
df_normal = df_normal.iloc[:, sensors]
df_attack=df_attack.iloc[:,sensors]

In [38]:
df_normal

,1_MV_001_STATUS
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
971938,2.0
971939,2.0
971940,2.0
971941,2.0


In [39]:
df_normal.columns

Index(['1_MV_001_STATUS'], dtype='object')

In [40]:
df_attack

,1_MV_001_STATUS
941103,1.0
941104,1.0
941105,1.0
941106,1.0
941107,1.0
...,...
971995,2.0
971996,2.0
971997,2.0
971998,2.0


In [25]:
df_attack['1_P_006_STATUS'].unique

<bound method Series.unique of 941103    1.0
941104    1.0
941105    1.0
941106    1.0
941107    1.0
         ... 
971995    1.0
971996    1.0
971997    1.0
971998    1.0
971999    1.0
Name: 1_P_006_STATUS, Length: 8465, dtype: float64>

In [10]:
sensorvalue=2

In [11]:
def calc_width_height(v):
    width = 2/np.sqrt(v) 
    return width
    
def draw_ellipse(i,j,centers,widths,X):
    plt.xlim(centers[i]-widths[i]/2,centers[i]+widths[i]/2)
    plt.ylim(centers[j]-widths[j]/2,centers[j]+widths[j]/2)
    ellipse = Ellipse(xy=(centers[i], centers[j]), width=widths[i], height=widths[j])
    ellipse.fill = False
    ellipse.set_ec('red')
    ax.add_patch(ellipse)
    plt.scatter(x = X[::,i], y = X[::,j])
    

In [12]:
X = df_normal.iloc[:,sensorvalue].values
X = hankel.fit(X,30,0.5)
X=X.T 
kmeans,optimal_k = cluster.fit(X)
print(optimal_k)
radiis = []
weights=[]
centers=[]
clusters_R=[]
clusters_V=[]
radius1=[]

for i in range(optimal_k):
    cluster_ = X[np.where(kmeans.labels_ == i)[0]]
    r = rank.fit(cluster_)
    clusters_R.append(r)
    print("r: "+str(r))
    U,Sigma,VT = np.linalg.svd(cluster_)
    V = VT.T
    clusters_V.append(V[:,:r])
    cluster_ = np.matmul(cluster_,V[:,:r])
    print(cluster_.shape)
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    widths=calc_width_height(weight)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    radius1.append(np.sqrt(var2))
    max_radius=math.sqrt(np.amax(var2))
    radiis.append(max_radius)
    
print(radiis)

# attack training


In [26]:
X_attack = df_attack.iloc[:,sensorvalue].values
X_att = hankel.fit(X_attack,30,0.5)
X_att=X_att.T 
print(X_att.shape)
radius2=[]
for i in range(optimal_k):
    cluster_=np.matmul(X_att,clusters_V[i])
    print(cluster_.shape)
    weight,center = meepc.fit(cluster_)
    weights.append(weight)
    centers.append(center)
    widths=calc_width_height(weight)
    var1=np.square(cluster_-center)
    var2=np.matmul(weight,var1.T)
    radius2.append(np.sqrt(var2))
radius2

(27, 600)
(27, 16)
(27, 0)
(27, 0)


[array([0.93959827, 0.93959827, 0.93959827, 0.93959827, 0.93959827,
        0.93959827, 0.93959827, 0.93959827, 0.93959827, 0.93959827,
        0.93959827, 0.93959827, 0.93959827, 0.93959827, 0.93959827,
        0.93959827, 0.93959827, 0.93959827, 0.93959827, 0.93959827,
        0.93959827, 0.97143373, 1.00000156, 0.98141691, 0.93959827,
        0.93959827, 0.93959827]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]

In [27]:
# X_attack = df_attack.iloc[:,sensorvalue].values
# X_att = hankel.fit(X_attack,10,0.5)
# X_att=X_att.T 
# kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
# kmeans.fit(X_att)
# radiis2=[]
# print(X_att.shape)
# radius2=[]
# for i in range(optimal_k):
#     cluster_ = X_att[np.where(kmeans.labels_ == i)[0]]
#     r = rank.fit(cluster_)
#     # clusters_R.append(r)
#     print("r: "+str(r))
#     U,Sigma,VT = np.linalg.svd(cluster_)
#     V = VT.T
#     clusters_V.append(V)
#     cluster_ = np.matmul(cluster_,V[:,:r])
#     print(cluster_.shape)
#     # cluster_=tsne.fit_transform(cluster_)
#     weight,center = meepc.fit(cluster_)
#     weights.append(weight)
#     centers.append(center)
#     widths=calc_width_height(weight)
#     var1=np.square(cluster_-center)
#     var2=np.matmul(weight,var1.T)
#     radius2.append(np.sqrt(var2))
#     # print(var2.shape)
#     max_radius=math.sqrt(np.amax(var2))
#     radiis2.append(max_radius)
# print(radiis2)

In [28]:
def threshold(radius1,radius2,optimal_k):
    threshold=[]
    radius3=[]
    mapping=[]
    argsort_indices=[]
    
    #appending radiis of positives and negatives in radius3
    for i in range(optimal_k):
        radius3.append(np.concatenate((radius1[i],radius2[i])))
    
    #created a mapping +1 for normal radius and -1 for attack radius
    for i in range(optimal_k):
        mapping.append(np.concatenate((np.ones(radius1[i].shape[0]),np.negative(np.ones(radius2[i].shape[0])))))
    
    #for each cluster finding threshold
    for i in range(optimal_k):
        argsort_indices.append(np.argsort(radius3[i]))
    for cluster in range(optimal_k): 
        positives=[]
        negatives=[]
        labels=[]
        for idx in argsort_indices[cluster]:
            labels.append(mapping[cluster][idx])
        neg=np.cumsum((1-np.array(labels))/2)
        pos=np.cumsum((1+np.array(labels))/2)
        no_pos=np.sum(1+np.array(labels)/2)
        fn=no_pos-pos

        #fscore
        final_radius=(2*pos)/(2*pos+fn+neg)
        # final_radius=(2/(3+(neg/pos)))
        # print(final_radius)   
        idx=np.argmax(final_radius)     
        radiii=radius3[cluster][argsort_indices[cluster][idx]]
        threshold.append(radiii)

    return threshold

In [29]:
# start=0
# for idx in argsort_indices[cluster]:
#     if(start==0 and mapping[cluster][idx]==1):
#         positives.append(1)
#         start=1
#     elif(start==0 and mapping[cluster][idx]==-1):
#         positives.append(0)
#         start=1
#     elif(mapping[cluster][idx]==-1):
#         positives.append(positives[-1])
#     elif(mapping[cluster][idx]==1):
#         positives.append(positives[-1]+1)

# start=0
# for idx in argsort_indices[cluster]:
#     if(start==0 and mapping[cluster][idx]==1):
#         negatives.append(0)
#         start=1
#     elif(start==0 and mapping[cluster][idx]==-1):
#         negatives.append(1)
#         start=1
#     elif(mapping[cluster][idx]==-1):
#         negatives.append(negatives[-1]+1)
#     elif(mapping[cluster][idx]==1):
#         negatives.append(negatives[-1])
# final_list=np.array(positives)-np.array(negatives)
# print(argsort_indices[cluster][np.argmax(final_list)])
# thresh=radius3[cluster][argsort_indices[cluster][np.argmax(final_list)]]
# threshold.append(thresh)

In [30]:
radius_cluster=threshold(radius1,radius2,optimal_k)

In [31]:
radius_cluster

[nan, 0.0, 0.0]

# testing 

In [32]:
df_test=pd.read_csv('~/data/testdata.csv')
df_test

,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LT_001_PV,1_MV_001_STATUS,1_MV_002_STATUS,1_MV_003_STATUS,...,3_AIT_001_PV,3_AIT_002_PV,3_AIT_003_PV,3_AIT_004_PV,3_AIT_005_PV,3_FIT_001_PV,3_LT_001_PV,LEAK_DIFF_PRESSURE,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
1,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
2,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
3,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
4,161.129,0.721464,11.7842,463.716,0.267211,1.890010,48.9329,2.0,1.0,1.0,...,0.0,8279.1,5.95083,743.356,0.594601,0.001333,64.5137,74.2107,1.92,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248395,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248396,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248397,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0
248398,176.467,0.613474,11.9281,430.680,0.228327,0.001214,66.1851,1.0,1.0,1.0,...,0.0,8279.1,13.83640,370.386,0.729505,0.595032,64.3879,66.1173,1.41,1.0


In [33]:
y_actual=df_test.iloc[:,-1]
# df_test_normal=df_test[df_test['Attack LABLE (1:No Attack, -1:Attack)']==1]
df_test=df_test.iloc[:,sensors]
df_test

,1_AIT_001_PV,1_MV_001_STATUS,1_P_006_STATUS,2_MCV_007_CO,2_MCV_101_CO,2_MCV_201_CO,2_MCV_301_CO,2_MCV_401_CO,2_MCV_601_CO
0,161.129,2.0,1.0,0.0,9.86244,7.75309,20.2651,12.6836,12.7701
1,161.129,2.0,1.0,0.0,9.86244,7.75309,20.2651,12.6836,12.7701
2,161.129,2.0,1.0,0.0,9.86244,7.75309,20.2651,12.6836,12.7701
3,161.129,2.0,1.0,0.0,9.86244,7.75309,20.2651,12.6836,12.7701
4,161.129,2.0,1.0,0.0,9.86244,7.75309,20.2651,12.6836,12.7701
...,...,...,...,...,...,...,...,...,...
248395,176.467,1.0,1.0,0.0,0.00000,13.14970,19.2170,0.0000,17.5298
248396,176.467,1.0,1.0,0.0,0.00000,13.14970,19.2170,0.0000,17.5298
248397,176.467,1.0,1.0,0.0,0.00000,13.14970,19.2170,0.0000,17.5298
248398,176.467,1.0,1.0,0.0,0.00000,13.14970,19.2170,0.0000,17.5298


In [34]:
labels=hankel.fit(np.array(y_actual),10,0.5)
labels

array([[ 1.,  1.,  1., ..., -1.,  1.,  1.],
       [ 1.,  1.,  1., ..., -1.,  1.,  1.],
       [ 1.,  1.,  1., ..., -1.,  1.,  1.],
       ...,
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [35]:
labels.shape

(600, 826)

In [36]:
isatt=np.any(labels<0,axis=0)
att_indices=np.where(isatt)[0]
att_indices

array([803, 804, 805, 806, 807, 808, 809, 820, 821, 822, 823])

In [37]:
X_test=df_test.iloc[:,sensorvalue].values
Xtest=hankel.fit(X_test,30,0.5)  #size -> 1800x274
Xtest=Xtest.T  #size -> 274x1800
print(Xtest.shape)
distances=[]
for i in range(optimal_k):
    Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) #size -> 274 x R
    var1=np.square(Xtest_cluster-centers[i])
    var2=np.matmul(weights[i],var1.T)
    distance=np.sqrt(var2)
    distances.append(distance)

(826, 600)


In [38]:
distances[0].shape #size (274,)

(826,)

In [39]:
# distances matrix
stacked_arr=np.vstack(distances)
distances_arr=np.transpose(stacked_arr)
distances_arr #-> 274 X 4

array([[nan,  0.,  0.],
       [nan,  0.,  0.],
       [nan,  0.,  0.],
       ...,
       [nan,  0.,  0.],
       [nan,  0.,  0.],
       [nan,  0.,  0.]])

In [40]:
radiis

[nan, 0.0, 0.0]

In [41]:
check_anamoly=radius_cluster-distances_arr     # positive = inside cluster | negative = outside cluster
check_anamoly

array([[nan,  0.,  0.],
       [nan,  0.,  0.],
       [nan,  0.,  0.],
       ...,
       [nan,  0.,  0.],
       [nan,  0.,  0.],
       [nan,  0.,  0.]])

In [42]:
check_anamoly.shape

(826, 3)

In [43]:
predicted_attack=[266,281,285,287]
for i in predicted_attack:
    print(distances_arr[i])
actual_attack=[803, 804, 805, 806, 807, 808, 809, 820, 821, 822, 823]
print("actual attack distances")
for id in actual_attack:
    print(distances_arr[id])

[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
actual attack distances
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]


radius-[1.000000930168902, 0.998157468152326, 0.9986001021045486, 1.0001279549432047]

In [44]:
predicted_attack=[266,281,285,287]
actual_attack=[803, 804, 805, 806, 807, 808, 809, 820, 821, 822, 823]
print("predicted attack")
for id in predicted_attack:
    print(check_anamoly[id])
print("actual attack")
for id in actual_attack:
    print(check_anamoly[id])

predicted attack
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
actual attack
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]
[nan  0.  0.]


In [45]:
#check for if any datapoint is coming inside the cluster
is_positive=np.any(check_anamoly>0,axis=1)
#indices of those rows
positive_indices=np.where(is_positive)[0]


In [46]:
for i in range(826):
  if i not in positive_indices:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

[803, 804, 805, 806, 807, 808, 809, 820, 821, 822, 823])

In [120]:
len(positive_indices)

822